# QuDiPy tutorial: Hubbard model charge stability diagrams

This tutorial will show how to use the charge stability module to generate a charge stability diagram using the constant interaction model and then perform the Hough transformation.

In order for the file to not be too large, the output of the cells has been cleared so you will need to run the cells on your local machine to see the output.

## 1. Add the current location to path
In order for this tutorial to work, you will need to run this from the tutorials folder.

In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

## 2. Import the relevant modules and set plotting parameters

In [ ]:
#From module
import qudipy.chargestability.csd_hubbard as hub
import qudipy.chargestability.csd_analysis as an

#For plotting
import matplotlib.pyplot as plt

#Makes plots bigger (optional)
%matplotlib inline 
plt.rcParams['figure.figsize'] = [12, 9]

## 3. Initialize the Hubbard model object

In order to initialize the charge stability object, we need to specify a few different parameters:  
- Number of sites: how many dots are in the system.  
- Number of electrons: maximum number of electrons that can be in the system at any point. Must not be greater than 2 times the number of sites since all electrons are taken to be in the ground state in this model.  
- Capacitance matrix: Dimensionless matrix with converts from gate voltages to chemical potentials on individual dots. Must be an (number of sites) by (number of sites) sized matrix.  
- Parameters: Dictionary which contains all the other parameters which define a particular quantum dot system. If a parameter that is expected is not specified, the code will treat it as 0 (this can be convenient in cases where some dots are decoupled from others in the system). There are two types of terms which are described below:
     - U_ij: Direct electrostatic repulsion between site i and j. If i=j, then this is considered to be the self repulsion on dot in i. Units are in meV.
     - t_ij: Tunnel coupling between site i and j. Will only be taken into account when a transition of this type (i.e without spin flip) is possible. Unit are in meV.



In [ ]:
# Number of sites
n_sites = 2
# Number of electrons
n_electrons = 4
# Pamameters
params = {'U_11':8.36 * 1E-3, 'U_22':9.38 * 1E-3, 'U_12':2.6 * 1E-3, 't_12':0.3 * 1E-3} 

## To generate the capacitance matrix

# These particular formulas for alpha_i are derived from the constant interaction model
alpha_1 = (params['U_22'] - params['U_12'])*params['U_11'] / (params['U_11']*params['U_22'] - params['U_12']**2)
alpha_2 = (params['U_11'] - params['U_12'])*params['U_22'] / (params['U_11']*params['U_22'] - params['U_12']**2)

cap_matrix = [[alpha_1,1-alpha_1],[1-alpha_2,alpha_2]]

# Generate charge stability diagram
hub_csd = hub.HubbardCSD(n_sites, n_electrons, cap_matrix, h_mu=True, h_t=True, h_u=True, **params)

## 4. Generate charge stability diagram

Now that we have initialized the object with all the parameters required, we can plot the charge stability diagram. In order to plot the charge stability diagram, we need to specify a few things as well:  
- Intital charge configuration: A list which specifies the initial voltage of all the gates in the system. Must be the same length as the number of dots in the system.
- Index of gate voltages to sweep: Since the charge stability diagram is a two dimensional plot with 2 gate voltages, we must specify which two voltages to sweep. In order to do this, we specify the index of the two voltages we want to sweep from the initial charge configuration.
- Final voltages of gates swept: These last two parameters specify the final voltages of two gates we wanted to sweep to generate the charge stability diagram.
  
This will not plot the charge stability diagram, only generate the data that populates it.
 

In [ ]:
hub_csd.generate_csd([-0.005,-0.005], 1, 2, 0.02, 0.02, num=75)

## 5. Plot charge stability diagram

Finally, we can plot the results of the charge stability diagram. In order to do this, we must pass it to the CSDAnalysis object. We can pass an optional parameter corresponding to a list of capacitances, which tells us how to convert from occupation in each dot to a current in order to replicate what an experiment might see. This list must be the same length as the number of sites since in general, each site will have a different coupling to a charge sensor. If no capacitances are passed, a unique color will be given to each occupation. However, this color is meaningless, so further analysis of the object cannot be done unless capacitances are passed.

In [ ]:
csd_an = an.CSDAnalysis(hub_csd, capacitances=[2.5,1.5])
csd_an.plot(derivative=True)